# 1. Récolte des données
La création de nos bases de données va se faire en plusieurs étapes. Dans un premier temps nous allons récupérer les données des trotinnettes présentes à Paris et celle des métro et rer à paris. Ensuite nous allons calculer la distance des trotinnettes aux moyens de transport en commun. Aussi nous allons récupérer toutes des données socio-démographique de la ville de Paris par quartier admnistratifs. Les deux bases finales contiennent pour l'une toutes les informations par quartiers admnistratifs et la seconde comptient les positions des trotinnettes associées aux informations du quartier admnistratifs dans lesquelles elles se trouvent. Nous vous reccommandons de ne pas éxécuter certainespartie de ce code car cela met plusieurs heures pour s'éxécuter totalement. Nous fournoussis les fichiers créés à partir de l'éxécution de ces parties. La récolte des données des trotinnettes s'est faite sur 2 jours (Lundi et Samedi) et plusieurs heures : 9h 14h et 19h pour le Lundi et 9h, 14h, 16h, 18h et 19h. Le choix d'avoir chois plus d'heures le Lundi se justifie par le fait que c'est le week-end et que tous les commerces sont ouverts, ce qui nous laisse présupposer qu'il y a plus de mobilité ce jour-là. Ainsi, nous fournissons un exemple du code pour la journée de Samedi.

In [8]:
path="/Users/dalilyoucefi/Documents/ProjetInfo/"

In [1]:
import requests
import pandas as pd
import geopandas as gpd

import tempfile
import zipfile
import math
import urllib.parse
import numpy as np
from urllib import request
import bs4

## 1.1 Récolte des données des trotinnettes (à ne pas éxécuter)

In [ ]:

## On récupère les données des trotinnettes sur l'API de Tier et on les met dans unre liste ppur créer le DataFrame
r=requests.get("https://platform.tier-services.io/v2/vehicle?zoneId=PARIS",headers={"X-Api-Key": "bpEUTJEBTf74oGRWxaIcW7aeZMzDDODe1yBoSxi2"})
print(r.json()["data"][1])
df= pd.DataFrame(data=r.json()["data"])
BatteryLevel=[r.json()["data"][i]["attributes"]["batteryLevel"] for i in range(len(r.json()["data"]))]
Lat=[r.json()["data"][i]["attributes"]["lat"] for i in range(len(r.json()["data"]))]
Lng=[r.json()["data"][i]["attributes"]["lng"] for i in range(len(r.json()["data"]))]
Type=[r.json()["data"][i]["attributes"]["vehicleType"] for i in range(len(r.json()["data"]))]
Status=[r.json()["data"][i]["attributes"]["state"] for i in range(len(r.json()["data"]))]
ID=[r.json()["data"][i]["id"] for i in range(len(r.json()["data"]))]



In [ ]:
# On indique aussi l'heure et le jour où on a récupéré les données
df_Tier=pd.DataFrame(columns=["ID","BatteryLevel","Lat","Lng","Type","Status"])
df_Tier = df_Tier.fillna(0)
df_Tier["ID"]=ID
df_Tier["BatteryLevel"]=BatteryLevel
df_Tier["Lat"]=Lat
df_Tier["Lng"]=Lng
df_Tier["Type"]=Type
df_Tier["Status"]=Status
df_Tier["Heure"]="19h"
df_Tier["Jour"]="Samedi"

In [ ]:
#On sauvegarde la base de donnée
df_Tier.to_csv(path+"TierBase19HSa.csv")

In [ ]:
#On importe les données géographique des quartiers administratifs et des arrondissements de Paris trouvé sur le site OpenDataParis
gdf = gpd.read_file(path+"arrondissements.geojson').sort_values(by="c_ar").reset_index()
gdf2=gpd.read_file(path+ "quartier_paris.geojson')


In [ ]:
#On transforme la base en GeoDataFrame
df_Tier=pd.read_csv(path+"TierBase19HSa.csv")
gdf_Tier = gpd.GeoDataFrame(df_Tier, geometry=gpd.points_from_xy(df_Tier.Lng, df_Tier.Lat))

In [ ]:
#On crée 2 fonctions qui permettent de déterminer dans quel quartier administratif et arrondissement se trouvent les trotinnettes 

def arrondissementappartenance(point):
    for i in range(20):
        if point.within(gdf["geometry"][i]):
            return int(i+1)


def quartierappartenance(point):
    for i in range(80):
        if point.within(gdf2["geometry"][i]):
            return gdf2["c_qu"][i]

On fusionne les bases sur les données récoltées dans la journée (fichiers pour les autres heures et jour fournis).

In [5]:
#On sauvegarde 
gdf_Tier["Arrondissement"]= gdf_Tier["geometry"].apply(arrondissementappartenance)
gdf_Tier["id_quartier"]=gdf_Tier["geometry"].apply(quartierappartenance)
gdf_Tier.head()

gdf_Tier.to_file(path+"GeoTier19HSa.geojson", driver='GeoJSON')

NameError: name 'gdf_Tier' is not defined

In [9]:
gdf = gpd.read_file(path+"GeoTier9HSa.geojson")
gdf2=gpd.read_file(path+"GeoTier14HSa.geojson")
gdf3=gpd.read_file(path+"GeoTier16HSa.geojson")
gdf4=gpd.read_file(path+"GeoTier18HSa.geojson")
gdf5=gpd.read_file(path+"GeoTier19HSa.geojson")
gdf6=gdf.append(gdf2,ignore_index=True)
gdf6=gdf6.append(gdf3,ignore_index=True)
gdf6=gdf6.append(gdf4,ignore_index=True)
gdf6=gdf6.append(gdf5,ignore_index=True)

gdf6.to_file(path+"GeoTierSa.geojson", driver='GeoJSON')




## 1.2 Récupération des positions des stations de métro, RER et Tramway et calcul de la distance de la station la plus proche à chaque trotinnette 

In [10]:
#On récupère les informations des stations de métro, RER et Tramway de Paris sur le site d'ile de ffrance mobilité
stations = open(path+"emplacement-des-gares-idf.geojson", "r")
stations = gpd.read_file(stations)
stations= stations[["gares_id","nomlong","mode","ligne","geometry"]]
stations.head()



,gares_id,nomlong,mode,ligne,geometry
0,93,BOTZARIS,Metro,7b,POINT (2.38912 48.87948)
1,30,ASSEMBLEE NATIONALE,Metro,12,POINT (2.32100 48.86079)
2,47,BALARD,Metro,8,POINT (2.27816 48.83593)
3,54,BASTILLE,Metro,1,POINT (2.36932 48.85248)
4,59,BEL-AIR,Metro,6,POINT (2.40092 48.84134)


In [11]:
#Comme les coordonnées sont en latitudes et longitudes, on ne peut pas calculer de distance, donc nous allons devoir créer une fonction pour la calculer

df1 = gpd.read_file(path+"GeoTierSa.geojson")
df1["x"]=df1["geometry"].x
df1["y"]=df1["geometry"].y
stations["x"]=stations["geometry"].x
stations["y"]=stations["geometry"].y




In [12]:
def distance(x1,y1,x2,y2):
    R = 6373.0
    
    
    lat1 = math.radians(y1)

    lon1 = math.radians(x1)
    lat2 = math.radians(y2)
    lon2 = math.radians(x2)
    
    dlon = lon2 - lon1
    
    dlat = lat2 - lat1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return (distance)

In [13]:
#On retrouve ainsi la station de métro la plus proche et toutes ses informations
min_distancei = float(1000000)
min_distance = []
station_min = []
ligne_min=[]

for i in range(len(df1)):
    min_distancei = float(100000)
    station_min0 = "Essai"
    ligne_min0="Essai"
    
    for j in range(len(stations)):
        
        if distance(df1["x"][i],df1["y"][i],stations["x"][j],stations["y"][j]) <= min_distancei :
                min_distancei = distance(df1["x"][i],df1["y"][i],stations["x"][j],stations["y"][j])
                station_min0 = stations['nomlong'][j]
                ligne_min0=stations["ligne"][j]
                
           

    min_distance.append(min_distancei)
    station_min.append(station_min0)
    ligne_min.append(ligne_min0)
    

In [15]:
#On ajoute ça au GeoDataFrame
df1["station_min"] = station_min
df1["min_distance"] = min_distance
df1["ligne_min"]=ligne_min

df1.head()

df1.to_file(path+"TierMetroSa.geojson", driver='GeoJSON')

## 1.3 Données démographique par quartiers admnistratifs  

In [2]:
#Nous allons récupérer sur le site de l'INSEE les données démographiques des arrondissements par tranche d'âge
liste_ageprop=[]
liste_ageabs=[]
for j in range(1,21):
    if j<10:
        arr="0"+str(j)
    else:
        arr=str(j)
    url_req="https://www.insee.fr/fr/statistiques/2011101?geo=COM-751"+arr
    request_text = request.urlopen(url_req).read()
    page = bs4.BeautifulSoup(request_text, "lxml")
    tableau_participants = page.find('table', {'id' : "produit-tableau-POP_T0"})
    table_body = tableau_participants.find('tbody')
    rows = table_body.find_all('tr')
    li1=[]
    li2=[]
    for i in range(len(rows)):
        if i>=2:
           
            cols = rows[i].find_all('td')
            cols = [ele.text.strip() for ele in cols]
            li1.append(cols[-1])
            li2.append(cols[-2])
    liste_ageprop.append(li1)
    liste_ageabs.append(li2)
        


KeyboardInterrupt: 

In [28]:
 #On va rajouter ces données dans la Dataframe des arrondissements     
gdf = gpd.read_file(path+"arrondissements.geojson").sort_values(by="c_ar").reset_index()

liste2=np.array(liste_ageprop)
gdf["prop_0-14"]=liste2[:,0].tolist()
gdf["prop_15-29"]=liste2[:,1].tolist()
gdf["prop_30-44"]=liste2[:,2].tolist()
gdf["prop_45-59"]=liste2[:,3].tolist()
gdf["prop_60-74"]=liste2[:,4].tolist()
gdf["prop_75+"]=liste2[:,5].tolist()

liste3=np.array(liste_ageabs)
gdf["nombre_0-14"]=liste3[:,0].tolist()
gdf["nombre_15-29"]=liste3[:,1].tolist()
gdf["nombre_30-44"]=liste3[:,2].tolist()
gdf["nombre_45-59"]=liste3[:,3].tolist()
gdf["nombre_60-74"]=liste3[:,4].tolist()
gdf["nombre_75+"]=liste3[:,5].tolist()

In [29]:
#On supprime les caractères en trop pouvoir convertir les données en flottant
for i in range(len(gdf)):

    gdf["prop_0-14"][i]=gdf["prop_0-14"][i].replace('\xa0','')
    gdf["prop_15-29"][i]=gdf["prop_15-29"][i].replace('\xa0','')
    gdf["prop_30-44"][i]=gdf["prop_30-44"][i].replace('\xa0','')
    gdf["prop_45-59"][i]=gdf["prop_45-59"][i].replace('\xa0','')
    gdf["prop_60-74"][i]=gdf["prop_60-74"][i].replace('\xa0','')
    gdf["prop_75+"][i]=gdf["prop_75+"][i].replace('\xa0','')
    gdf["nombre_0-14"][i]=gdf["nombre_0-14"][i].replace('\xa0','')
    gdf["nombre_15-29"][i]=gdf["nombre_15-29"][i].replace('\xa0','')
    gdf["nombre_30-44"][i]=gdf["nombre_30-44"][i].replace('\xa0','')
    gdf["nombre_45-59"][i]=gdf["nombre_45-59"][i].replace('\xa0','')
    gdf["nombre_60-74"][i]=gdf["nombre_60-74"][i].replace('\xa0','')
    gdf["nombre_75+"][i]=gdf["nombre_75+"][i].replace('\xa0','')
    gdf["prop_0-14"][i]=gdf["prop_0-14"][i].replace(',','.')
    gdf["prop_15-29"][i]=gdf["prop_15-29"][i].replace(',','.')
    gdf["prop_30-44"][i]=gdf["prop_30-44"][i].replace(',','.')
    gdf["prop_45-59"][i]=gdf["prop_45-59"][i].replace(',','.')
    gdf["prop_60-74"][i]=gdf["prop_60-74"][i].replace(',','.')
    gdf["prop_75+"][i]=gdf["prop_75+"][i].replace(',','.')
    gdf["nombre_0-14"][i]=gdf["nombre_0-14"][i].replace(',','.')
    gdf["nombre_15-29"][i]=gdf["nombre_15-29"][i].replace(',','.')
    gdf["nombre_30-44"][i]=gdf["nombre_30-44"][i].replace(',','.')
    gdf["nombre_45-59"][i]=gdf["nombre_45-59"][i].replace(',','.')
    gdf["nombre_60-74"][i]=gdf["nombre_60-74"][i].replace(',','.')
    gdf["nombre_75+"][i]=gdf["nombre_75+"][i].replace(',','.')
gdf["prop_0-14"]=(gdf["prop_0-14"]).astype(float)
gdf["prop_15-29"]=gdf["prop_15-29"].astype(float)
gdf["prop_30-44"]=gdf["prop_30-44"].astype(float)
gdf["prop_45-59"]=gdf["prop_45-59"].astype(float)
gdf["prop_60-74"]=gdf["prop_60-74"].astype(float)
gdf["prop_75+"]=gdf["prop_75+"].astype(float)
gdf["nombre_0-14"]=gdf["nombre_0-14"].astype(float)
gdf["nombre_15-29"]=gdf["nombre_15-29"].astype(float)
gdf["nombre_30-44"]=gdf["nombre_30-44"].astype(float)
gdf["nombre_45-59"]=gdf["nombre_45-59"].astype(float)
gdf["nombre_60-74"]=gdf["nombre_60-74"].astype(float)
gdf["nombre_75+"]=gdf["nombre_75+"].astype(float)

/Users/dalilyoucefi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/dalilyoucefi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/dalilyoucefi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
#On va rajouter la population totale et ne garder que les informations essentielles
gdf["pop"]=gdf["nombre_0-14"]+gdf["nombre_15-29"]+gdf["nombre_30-44"]+gdf["nombre_45-59"]+gdf["nombre_60-74"]+gdf["nombre_75+"]
li=[9863,21872,29638,17838,24817,21062,13936,10147,26888,323888,41341,8686,24896,23684,27201,9714,29166,31701,27274,32494]
gdf["densite"]=li
gdf2=gdf[["l_ar","c_ar","prop_0-14","prop_15-29",'prop_30-44', 'prop_45-59', 'prop_60-74', 'prop_75+', 'nombre_0-14',
       'nombre_15-29', 'nombre_30-44', 'nombre_45-59', 'nombre_60-74',
       'nombre_75+', 'pop', 'densite']]



In [31]:
#On fusionne la base avec celle des quartiers administratifs
gdf_quart=gpd.read_file(path+"quartier_paris.geojson")
gdf3=gdf_quart.merge(gdf2, left_on="c_ar",right_on="c_ar")

In [32]:
# On va aussi rajouter les données du nombre de transports en commun dans les quartiers administratifs
stations = open(path+"emplacement-des-gares-idf.geojson", "r")
stations = gpd.read_file(stations)
stations= stations[["gares_id","nomlong","mode","ligne","geometry"]]



In [33]:
def nombrestation(x):
    li=[0 for i in range(80)]
    for j in range(len(x)):
        
        for i in range(80):
            if x["geometry"][j].within(gdf3["geometry"][i]):
                li[i]+=1
    return (li)

gdf3["nombretransport"]=nombrestation(stations)

In [34]:
gdf3.head()

,n_sq_qu,n_sq_ar,c_qu,surface,l_qu,perimetre,c_quinsee,c_ar,geometry,l_ar,...,prop_75+,nombre_0-14,nombre_15-29,nombre_30-44,nombre_45-59,nombre_60-74,nombre_75+,pop,densite,nombretransport
0,750000020,750000005,20,433197.792441,Sorbonne,2892.944068,7510504,5,"POLYGON ((2.34924 48.84452, 2.34888 48.84449, ...",5ème Ardt,...,9.5,6735.0,17195.0,10190.0,9966.0,9148.0,5615.0,58849.0,24817,4
1,750000019,750000005,19,703631.162923,Val-de-Grâce,3485.831358,7510503,5,"POLYGON ((2.34548 48.84507, 2.34661 48.84480, ...",5ème Ardt,...,9.5,6735.0,17195.0,10190.0,9966.0,9148.0,5615.0,58849.0,24817,1
2,750000017,750000005,17,604156.072897,Saint-Victor,3087.215040,7510501,5,"POLYGON ((2.35492 48.84378, 2.35492 48.84364, ...",5ème Ardt,...,9.5,6735.0,17195.0,10190.0,9966.0,9148.0,5615.0,58849.0,24817,3
3,750000018,750000005,18,798389.398463,Jardin-des-Plantes,4052.729521,7510502,5,"POLYGON ((2.36456 48.84366, 2.36450 48.84358, ...",5ème Ardt,...,9.5,6735.0,17195.0,10190.0,9966.0,9148.0,5615.0,58849.0,24817,2
4,750000033,750000009,33,717091.569643,Saint-Georges,3429.188334,7510901,9,"POLYGON ((2.33923 48.87679, 2.33816 48.87686, ...",9ème Ardt,...,6.0,8367.0,14887.0,15117.0,10598.0,7031.0,3556.0,59556.0,26888,4


In [35]:
#On sauvegarde

gdf3.to_file(path+"BaseQuartier.geojson",driver='GeoJSON')

## 1.4 Logements sociaux et commerces à Paris

In [37]:
#importation des données des logements sociaux à Paris
gdf_logements_soc = gpd.read_file(path+"logements-sociaux-finances-a-paris.geojson")



In [38]:
#suppression de variables
gdf_logements_soc.drop(['nature_programme','arrdt','ville','n_livraison','adresse_programme','code_postal','annee','bs','nb_plai','nb_plus','nb_pluscd','nb_pls','mode_real','commentaires','coord_x_l93','coord_y_l93'], 1, inplace=True)



In [39]:
# on importe les données
file_name='quartier_paris.geojson'
file = open(file_name)
gdf = gpd.read_file(file)



In [40]:
def quartierappartenance(point):
    for i in range(80):
        if point.within(gdf["geometry"][i]):
            return gdf["l_qu"][i]

def numquartierappartenance(point):
    for i in range(80):
        if point.within(gdf["geometry"][i]):
            return gdf["c_qu"][i]
gdf_logements_soc["quartier"]=gdf_logements_soc["geometry"].apply(quartierappartenance)
gdf_logements_soc["numquartier"]=gdf_logements_soc["geometry"].apply(numquartierappartenance)
gdf_logements_soc

,nb_logmt_total,geometry,quartier,numquartier
0,12,POINT (2.35610 48.88885),Goutte-d'Or,71
1,22,POINT (2.35272 48.89786),Goutte-d'Or,71
2,37,POINT (2.28424 48.85746),Muette,62
3,16,POINT (2.30706 48.83217),Saint-Lambert,57
4,14,POINT (2.33014 48.88459),Grandes-Carrières,69
...,...,...,...,...
3579,8,POINT (2.35167 48.88680),Goutte-d'Or,71
3580,11,POINT (2.35457 48.85843),Saint-Merri,13
3581,26,POINT (2.37470 48.86153),Saint-Ambroise,42
3582,22,POINT (2.35424 48.85889),Saint-Merri,13


In [45]:
# on va indiquer le nombre de logement qu'on a par quartier administratif

gdf_logements_sociaux = gdf_logements_soc.groupby(['numquartier'],as_index=False).sum()
gdf_logements_sociaux.columns

Index(['numquartier', 'nb_logmt_total'], dtype='object')

In [46]:
# on importe les données
gdf_l = gpd.read_file(path+"logement-encadrement-des-loyers.geojson")

In [47]:
# suppression de variables
gdf_l.drop(['ville','epoque','meuble_txt','code_grand_quartier','piece','max','min','id_zone','annee'],1,inplace=True)


In [48]:
# on va calculer la moyenne des loyers dans chaque quartier administratif
gdf_loyer=gdf_l.groupby(['id_quartier'],as_index=False).mean()
g1 = gdf_loyer.rename(columns={'id_quartier':'numquartier'})

In [52]:
# on importe les données sur les commerces parisiens
file_name=path+ 'coronavirus-commercants-parisiens-livraison-a-domicile.geojson'
file = open(file_name)
gdf_co = gpd.read_file(file)

gdf_co['quartier']=0

In [53]:
# on va indiquer à quel quartier administratif appartient le commerce 
gdf_co["quartier"]=gdf_co["geometry"].apply(quartierappartenance)
gdf_co['numquartier']=gdf_co['geometry'].apply(numquartierappartenance)

In [54]:
# on va supprimer des variables 
gdf_co.drop(['code_postal','description','nom_du_commerce','adresse','telephone','services','mail','type_de_commerce','site_internet','precisions'],1,inplace=True)

In [55]:

# on va créer une variable qui va indiquer le nombre total de commerce par quartier administratif
gdf_co['nombre_de_commerce']=1
gdf_commerce=gdf_co.groupby('numquartier',as_index=False).sum()
gdf_commerce

df1=pd.merge(g1,gdf_logements_sociaux,on="numquartier")

df_final = pd.merge(df1,gdf_commerce,on="numquartier")

df_final.to_csv(path+"bdd_quart.csv")

## 1.5 Données des entreprises  (à ne pas éxécuter)

In [ ]:
#On récolte les données des entreprises à partir de la base du site DatainfoGreffe

ent=pd.read_csv("/Users/dalilyoucefi/Downloads/entreprise.csv", sep=",")
ent["Code postal"]=ent["Code postal"].astype(str)
ent["fulladress"]=ent["Adresse"]+", "+ent["Code postal"]+ " "+ent["Ville"]
ent=ent.dropna().reset_index()
#On supprime cette ligne car elle pose un problème pour plus tard
ent=ent.drop([13522]).reset_index()




In [ ]:
#On va à partir de l'adresse récupérer les coordonnées 

def positionlon(adr):

    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
    api_url = "https://api-adresse.data.gouv.fr/search/?q="

    r = requests.get(api_url + urllib.parse.quote(adr),headers=headers)


    
    return(r.json()["features"][0]["geometry"]["coordinates"][0])

li=[]
i=0
for j in range(len(ent)):
    i+=1
    li.append(positionlon(ent["fulladress"][j]))
    print(i)

ent["lon"]=li
ent.to_csv(path+"entrepriselon.csv")
ent3=pd.read_csv(path+"entrepriselon.csv")


def positionla(adr):

    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
    api_url = "https://api-adresse.data.gouv.fr/search/?q="

    r = requests.get(api_url + urllib.parse.quote(adr),headers=headers)


    
    return(r.json()["features"][0]["geometry"]["coordinates"][1])

li1=[]
i=0
for j in range(len(ent3)):
    i+=1
    li1.append(positionla(ent3["fulladress"][j]))
    print(i)
ent3["la"]=li1

#On garde ces données car elles mettent beaucoup de temps à être récoltée
ent3.to_csv(path+"entrepriselonlat.csv")

In [ ]:
#On convertit en DataFrame
geoent = gpd.GeoDataFrame(ent3, geometry=gpd.points_from_xy(ent3.lon, ent3.la))
gdf2=gpd.read_file(path+"quartier_paris.geojson")

In [ ]:
#On détermine dans quel quartier se trouve chaque entreprise
def quartierappartenance(point):
    for i in range(80):
        print(i)
        if point.within(gdf2["geometry"][i]):
            return gdf2["l_qu"][i]
def quartierappartenance2(point):
    for i in range(80):
        print(i)
        if point.within(gdf2["geometry"][i]):
            return gdf2["c_qu"][i]
        
geoent["nomquartier"]=geoent["geometry"].apply(quartierappartenance)
geoent["id_quartier"]=geoent["geometry"].apply(quartierappartenance2)



In [ ]:
#On fait un groupby pour calculer la moyenne de chiffre d'affaire et par quartier
geoent['nombreent']=1
ent1=geoent.groupby('id_quartier',as_index=False).sum()
ent1=ent1[["id_quartier","nombreent"]]

ent2=geoent.groupby('id_quartier',as_index=False).mean()
ent2=ent2[["id_quartier","CA 1"]]
df1=pd.merge(ent1,ent2,on="id_quartier")


In [ ]:
#On sauvegarde
df1.to_csv(path+"entreprisefinal.csv")

## 1.6 Fusion de toutes les bases pour en avoir 2 (Pour Samedi)

Index(['n_sq_qu', 'n_sq_ar', 'c_qu', 'surface', 'l_qu', 'perimetre',
       'c_quinsee', 'c_ar', 'l_ar', 'prop_0-14', 'prop_15-29', 'prop_30-44',
       'prop_45-59', 'prop_60-74', 'prop_75+', 'nombre_0-14', 'nombre_15-29',
       'nombre_30-44', 'nombre_45-59', 'nombre_60-74', 'nombre_75+', 'pop',
       'densite', 'nombretransport', 'geometry'],
      dtype='object')

In [76]:
#On charge toutes les bases sauvegardées pour les fusionner afin d'avoir seulement 2 bases de travail
gdf_quart1= gpd.read_file(path+"BaseQuartier.csv")[["c_qu","l_qu","c_ar","l_ar", 'prop_0-14', 'prop_15-29', 'prop_30-44',
       'prop_45-59', 'prop_60-74', 'prop_75+', 'nombre_0-14', 'nombre_15-29',
       'nombre_30-44', 'nombre_45-59', 'nombre_60-74', 'nombre_75+', 'pop', 'densite', 'nombretransport']]
gdf_quart2= gpd.read_file(path+"bdd_quart.csv")
gdf_ent=gpd.read_file(path+"entreprisefinal.csv")
gdf_Tier=gpd.read_file(path+"TierMetroSa.geojson").dropna()
gdf_quart1["c_qu"]=gdf_quart1["c_qu"].astype(str)
gdf_Tier["id_quartier"]=gdf_Tier["id_quartier"].astype(int)
gdf_Tier["id_quartier"]=gdf_Tier["id_quartier"].astype(str)
gdf_ent["id_quartier"]=gdf_ent["id_quartier"].astype(float)
gdf_ent["id_quartier"]=gdf_ent["id_quartier"].astype(int)
gdf_ent["id_quartier"]=gdf_ent["id_quartier"].astype(str)
gdf_ent=gdf_ent[["id_quartier","nombreent","CA 1"]]







In [77]:
#On fusionne toutes les données sur les quartiers
gdf2=gdf_quart1.merge(gdf_quart2, left_on="c_qu",right_on="numquartier")
gdf2=gdf2[['c_qu', 'l_qu', 'c_ar', 'l_ar', "prop_0-14","prop_15-29",'prop_30-44', 'prop_45-59',
       'prop_60-74', 'prop_75+', 'nombre_0-14', 'nombre_15-29', 'nombre_30-44',
       'nombre_45-59', 'nombre_60-74', 'nombre_75+', 'pop', 'densite',
       'nombretransport', 'field_1', 'numquartier', 'ref', 'nb_logmt_total',
       'nombre_de_commerce']]
gdf3=gdf2.merge(gdf_ent, left_on="c_qu",right_on="id_quartier")

In [78]:
#On rajoute les données des quartiers sur toutes les trotinnettes pour rendre le traitement plus simple
gdf_Tierfin=gdf_Tier.merge(gdf3, left_on="id_quartier",right_on="c_qu")
gdf_Tierfin=gdf_Tierfin[[ 'ID', 'BatteryLevel', 'Lat', 'Lng', 'Type', 'Status',
       'Heure', 'Jour', 'Arrondissement', 'densité', 'nomquartier', 'id_quartier_x', 'station_min', 'min_distance', 'ligne_min', 
       'geometry', 'c_qu', 'l_qu', 'c_ar', 'l_ar',"prop_0-14","prop_15-29", 'prop_30-44', 'prop_45-59',
       'prop_60-74', 'prop_75+', 'nombre_0-14', 'nombre_15-29', 'nombre_30-44',
       'nombre_45-59', 'nombre_60-74', 'nombre_75+', 'pop', 'densite',
       'nombretransport', 'field_1', 'numquartier', 'ref', 'nb_logmt_total',
       'nombre_de_commerce']]

In [79]:

gdf_Tierfin.rename(columns={"id_quartier_x":"id_quartier"},inplace=True)

#On sauvegarde cette base
gdf_Tierfin.to_file(path+"Tierbasefin1geoSa.geojson", driver='GeoJSON')
#On va compter cette le nombre de trotinnette présente en fonction de chaque heure dans chaque quartier
gdf_Tier['nombrtrot']=1

In [82]:

trot9=gdf_Tier[gdf_Tier["Heure"]=="9h"].groupby('id_quartier',as_index=False).sum()
trot9=trot9[["id_quartier","nombrtrot"]]

#  il y avait un quartier où aucune trotinette était présente, il faut donc rajouter une ligne  pour indiquer qu'il n'y en a pas

data=[("16",0)]
index=["1"]
columns=["id_quartier","nombrtrot"]
li=pd.DataFrame(data=data, index=index,columns=columns)

trot9=trot9.append(li,ignore_index=True)

trot9.rename(columns={"id_quartier":"id_quartier", "nombrtrot":"nombretrot9h"},inplace=True)
trot14=gdf_Tier[gdf_Tier["Heure"]=="14h"].groupby('id_quartier',as_index=False).sum()
trot14=trot14[["id_quartier","nombrtrot"]]
trot14.rename(columns={"id_quartier":"id_quartier", "nombrtrot":"nombretrot14h"},inplace=True)
trot19=gdf_Tier[gdf_Tier["Heure"]=="19h"].groupby('id_quartier',as_index=False).sum()
trot19=trot19[["id_quartier","nombrtrot"]]
trot19.rename(columns={"id_quartier":"id_quartier", "nombrtrot":"nombretrot19h"},inplace=True)
trot16=gdf_Tier[gdf_Tier["Heure"]=="16h"].groupby('id_quartier',as_index=False).sum()
trot16=trot16[["id_quartier","nombrtrot"]]
trot16.rename(columns={"id_quartier":"id_quartier", "nombrtrot":"nombretrot16h"},inplace=True)
trot18=gdf_Tier[gdf_Tier["Heure"]=="18h"].groupby('id_quartier',as_index=False).sum()
trot18=trot18[["id_quartier","nombrtrot"]]
trot18.rename(columns={"id_quartier":"id_quartier", "nombrtrot":"nombretrot18h"},inplace=True)
gdf4=gdf3.merge(trot9, left_on="c_qu",right_on="id_quartier")
gdf4=gdf4.merge(trot14, left_on="c_qu",right_on="id_quartier")
gdf4=gdf4.merge(trot19, left_on="c_qu",right_on="id_quartier")
gdf4=gdf4.merge(trot16, left_on="c_qu",right_on="id_quartier")
gdf4=gdf4.merge(trot18, left_on="c_qu",right_on="id_quartier")
gdf4["nombretrottotal"]=gdf4["nombretrot9h"]+gdf4["nombretrot14h"]+gdf4["nombretrot19h"]+gdf4["nombretrot18h"]+gdf4["nombretrot16h"]




In [83]:
#On sauvegarde enfin
gdf4.to_csv(path+"BaseQuartierSa.csv")

In [86]:
#On a fait de même pour Lundi On va fusionner les bases

df=pd.read_csv(path+"BaseQuartierSa.csv")
df2=pd.read_csv(path+"BaseQuartierLu.csv")
df["Jour"]="Samedi"
df2["Jour"]="Lundi"
df3=df.append(df2,ignore_index=True)

df3=df3[[ "Jour",'c_qu', 'l_qu', 'prop_0-14', 'prop_15-29',
       'prop_30-44', 'prop_45-59', 'prop_60-74', 'prop_75+', 'nombre_0-14',
       'nombre_15-29', 'nombre_30-44', 'nombre_45-59', 'nombre_60-74', 'nombre_75+',
       'pop', 'densite', 'nombretransport',  'numquartier', 'ref',
       'nb_logmt_total', 'nombre_de_commerce',  'nombreent',
       'CA 1', 'nombretrottotal']]

df3.to_csv(path+"BaseQuartierLuSa.csv")



gdf1=gpd.read_file(path+"Tierbasefin1geoLu.geojson")

gdf2=gpd.read_file(path+"Tierbasefin1geoSa.geojson")

gdf3=gdf1.append(gdf2,ignore_index=True)

gdf3.to_file(path+"GeoTierLunsSA.geojson", driver='GeoJSON')